In [ ]:
# credits:
# https://www.kaggle.com/code/olyatsimboy/aimo-openmath-mistral-baseline
# https://www.kaggle.com/code/aatiffraz/prompt-prediction-w-mixtral-mistral7b-gemma-llama
# https://www.kaggle.com/code/thedrcat/aimo-mixtral-baseline

**⚠️ Warning: Potential Eligibility Issue for AIMO Competition**

Please be aware that this notebook may not be eligible for submission to the AIMO competition. It utilizes the DeepSeekMath model and the vLLM framework, some components of which were released after the competition's stipulated deadline of February 23rd. Use of these updated resources could be considered non-compliant with the competition rules, which require all tools and datasets to be publicly available before the deadline. If you are planning to use this notebook or its methods as part of your competition submission, verify the eligibility with the competition organizers to avoid disqualification.

**Introduction to Tree of Thought (ToT) for Enhanced Reasoning in Language Models**

The Tree of Thought (ToT) is an advanced approach to reasoning with large language models, improving upon traditional methods such as greedy decoding or beam search. While these conventional methods might produce quick solutions, they often fall short in handling complex reasoning tasks effectively. ToT addresses this by exploring a diverse set of reasoning pathways, systematically evaluating them to discern the most accurate and reliable outcomes.

**Incorporation of Process Reward Model (PRM)**

A key component in this enhanced reasoning framework is the integration of a Process Reward Model (PRM). This model acts as a verifier for each reasoning step generated by the language model within the Tree of Thought. The PRM assesses the validity and logical consistency of each step, using predefined criteria to award positive or negative scores based on the appropriateness of the responses. This mechanism ensures that only the most promising reasoning paths are expanded, enhancing the overall quality and reliability of the solutions.

**Fallback to Python-Coding Based Self-Consistency**

In instances where the PRM does not confirm the validity of the reasoning paths, the framework falls back on a Python-coding based self-consistency approach. This secondary layer involves generating multiple diverse answers through code execution, then aggregating these answers to establish a consensus. This method, detailed in the Self-Consistency in Chains of Thought (SC-CoT) paper, serves as a reliable backup, ensuring that the final answers are not only generated but also validated through practical execution.

**Utilizing DeepSeekMath-7B RL-Tuned Model**

We employ the DeepSeekMath-7B model, which is fine-tuned with reinforcement learning techniques to enhance its performance on mathematical reasoning tasks. The model's advanced capabilities allow it to produce consistent reasoning paths and significantly reduce arithmetic hallucinations. By integrating the ToT approach with the PRM verification, we harness this model's potential to generate and refine diverse reasoning paths, leading to high accuracy and reliability in solutions.

**Practical Application in This Kernel**

This kernel will demonstrate how the ToT methodology, supported by PRM verification and Python-coding based self-consistency, can be effectively applied using the DeepSeekMath-7B model. Through practical examples and detailed code implementations, we aim to show a marked improvement in solving complex reasoning problems, particularly in mathematical contexts. This exploration provides valuable theoretical insights and practical tools for researchers and practitioners aiming to leverage advanced AI reasoning strategies in their work.


In [ ]:
!pip uninstall -y torch
!pip install --no-index --find-links=/kaggle/input/vllm-whl -U vllm

### vLLM Setup

- **vLLM (LLM Instance)**: The `LLM` class instance is configured to use a model located at `/kaggle/input/deepseek-math`. This setup involves several parameters optimized for performance:
  - **dtype='half'**: This setting uses 16-bit floating-point numbers (float16) instead of the standard 32-bit. This can significantly reduce memory usage and potentially speed up computations, albeit at the risk of lower numerical precision.
  - **enforce_eager=True**: Typically, this would enforce eager execution over graph-based execution, making debugging easier and operations more transparent but potentially at a cost to performance.
  - **gpu_memory_utilization=0.99**: This setting maximizes GPU memory usage to optimize the computation workload, allowing almost the entire GPU memory to be utilized.
  - **swap_space=4**: Allocates kv cache offload to system RAM, useful in handling larger models or batches that exceed physical GPU memory. May not be used during nrmal operation.
  - **max_model_len=2048**: Sets the maximum sequence length that the model will process, limiting the scope of inputs to ensure stable performance.
  - **kv_cache_dtype="fp8_e5m2"**: Utilizes an 8-bit floating-point format for storing key/value pairs in attention mechanisms, trading off precision for speed and memory efficiency. Necessary for generating a sequence of more than 1000 tokens in a single T4.
  - **tensor_parallel_size=1**: Operations are confined to a single GPU.

### Tokenizer and Model Configuration

- **Tokenization**: The tokenizer associated with the vLLM model is loaded to handle input preprocessing, which is crucial for correctly formatting and encoding the inputs into a form suitable for model processing.
- **PRM Tokenizer and Model**: A separate tokenizer and model (`AutoModelForCausalLM`) are loaded from a pretrained path. These are specifically used for a Process Reward Model (PRM), designed to evaluate generated responses at each reasoning step based on the presence of specific tokens (good or bad indicators):
  - **prm_candidate_tokens**: Encodes specific tokens that indicate positive and negative outcomes, aiding in evaluating the output from the reward model.
  - **step_tag_id**: Represents a specific token used to mark steps or stages in the reasoning process.

### Generation Speed and Determinism

- **Generation Speed**: vLLM is designed to be a high-speed model, potentially sacrificing some deterministic behaviors (like exact reproducibility of results) for faster response generation. This trade-off is particularly evident in environments where quick response generation is prioritized over absolute consistency. Setting a seed might not ensure reproducible outcomes.

### Operational Context

- vLLM is designed to run in it's own Docker environment, which can encapsulate its dependencies and settings. Running in a notebook environment requires some hacky solutions to load and manage models effectively, especially when interfacing with system resources or other software components.

In [ ]:
from vllm import LLM, SamplingParams
import pandas as pd
from tqdm import tqdm
import gc
import re
import sys
import subprocess
from collections import defaultdict, Counter
import numpy as np
from transformers import (AutoModelForCausalLM,
    AutoTokenizer,
    set_seed)
import torch
import math

llm = LLM(model="/kaggle/input/deepseek-math",
          dtype='half',
          enforce_eager=True,
          gpu_memory_utilization=0.99,
          swap_space=4,
          max_model_len=2048,
          kv_cache_dtype="fp8_e5m2",
          tensor_parallel_size=1)

tokenizer = llm.get_tokenizer()

good_token = '+'
bad_token = '-'
step_tag = 'ки'

prm_tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm')
prm_candidate_tokens = prm_tokenizer.encode(f"{good_token} {bad_token}")[1:] # [648, 387]
step_tag_id = prm_tokenizer.encode(f"{step_tag}")[-1] # 12902
prm_model = AutoModelForCausalLM.from_pretrained('/kaggle/input/math-shepherd-mistral-7b-prm',
                                                 torch_dtype=torch.float16,
                                                 device_map="balanced_low_0").eval()

In [ ]:
import pandas as pd
from tqdm import tqdm
PRIVATE = True

df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/test.csv')
df.head()

In [ ]:
if len(df) < 5:
    #df = pd.read_csv('/kaggle/input/ai-mathematical-olympiad-prize/train.csv')
    PRIVATE = False
df.head()

In [ ]:
def naive_parse(answer):
    out = []
    start = False
    end = False
    for l in reversed(list(answer)):
        if l in '0123456789' and not end:
            start = True
            out.append(l)
        else:
            if start:
                end = True
        
    out = reversed(out)
    return ''.join(out)

In [ ]:
import re
import sys
import os
import subprocess

def top_n_strings(input_list, n):
    # Sort the list based on the prob values in descending order
    sorted_list = sorted(input_list, key=lambda x: x[1], reverse=True)

    # Get the top n elements
    top_n = sorted_list[:n]

    # Extract the strings from the top n elements
    result = [item[0] for item in top_n]

    return result


def process_output(output):
    result = output
    
    try:
        code = output.split('```')[1][7:]

        with open('code.py', 'w') as fout:
            fout.write(code)

        batcmd = 'timeout 7 ' + sys.executable + ' code.py'
        try:
            shell_output = subprocess.check_output(batcmd, shell=True).decode('utf8')
            print(shell_output)
            code_output = round(float(eval(shell_output))) % 1000
        except:
            code_output = -1
            
        if os.path.exists('code.py'):
            os.remove('code.py')

        print('CODE RESULTS', code_output)
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        code_output = -1
    
    try:
        result_output = re.findall(r'\\boxed\{(.*)\}', result)

        print('BOXED', result_output)
        if not len(result_output):
            result_output = naive_parse(result)
        else:
            result_output = result_output[-1]

        print('BOXED', result_output)
        if not len(result_output):
            result_output = -1
        
        else:
            result_output = round(float(eval(result_output))) % 1000
    
    except Exception as e:
        print(e)
        print('ERROR PARSING')
        result_output = -1
    
    return result_output, code_output

In [ ]:
batch_size = 1

def eval_prm(candidates):
    # Initialize a list to store all the log probabilities
    all_log_probs = []

    # Process the candidates in batches
    for i in range(0, len(candidates), batch_size):
        # Select a batch of candidates
        batch_candidates = candidates[i:i + batch_size]

        # Encode all candidates into a batch of input IDs
        encoded_inputs = [prm_tokenizer.encode(candidate, return_tensors="pt") for candidate in batch_candidates]

        # Pad the encoded inputs to the same length
        max_length = max([input_id.shape[1] for input_id in encoded_inputs])  # Find the longest sequence
        padded_inputs = [
            torch.nn.functional.pad(input_id, (0, max_length - input_id.size(1)), value=prm_tokenizer.pad_token_id) for
            input_id in encoded_inputs]
        input_ids = torch.cat(padded_inputs, dim=0).to("cuda:1")  # Concatenate the padded inputs into a tensor

        with torch.no_grad():
            logits = prm_model(input_ids).logits[:, :, prm_candidate_tokens]

            scores = logits.softmax(dim=-1)[:, :, 0].squeeze()

            # Extract log probabilities for the specific candidate tokens
            log_probs = scores.log()

            if batch_size == 1:
                batch_log_probs = log_probs[-1].flatten()
            else:
                batch_log_probs = log_probs[:, -1].flatten()

            # Collect the log probabilities from this batch
            all_log_probs.extend(batch_log_probs.cpu().tolist())

    return all_log_probs

In [ ]:
stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
stop_words.append("\n")

tool_stop_words = [tokenizer.eos_token if tokenizer is not None and tokenizer.eos_token is not None else '</s>']
tool_stop_words.append("```output")

sampling_params = SamplingParams(temperature=1.0,
                                 max_tokens=256,
                                 stop=stop_words)

tool_sampling_params = SamplingParams(temperature=1.0,
                                      max_tokens=2048,
                                      stop=tool_stop_words)

cot_instruction = "\nPlease reason step by step, and put your final answer within \\boxed{}."

tool_instruction = '\nPlease integrate natural language reasoning with programs to solve the problem above, and put your final answer within \\boxed{}.'
n = 1
all_prompts = []
total_results = []
total_answers = []

for i in tqdm(range(len(df))):
    id_ = df['id'].loc[i]
    problem = df['problem'].loc[i]

    messages = [
        {
            "role": "user",
            "content": problem + cot_instruction
        }
    ]

    base_prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False
    )
    current_level = 0

    current_level_nodes = [(base_prompt, 0)]  # Tuple of (node, cumulative_logprob)
    completed_paths = []

    while len(completed_paths) < n:
        # Prepare batch for generation
        batch_prompts = [node for node, _ in current_level_nodes]
        batch_responses = llm.generate(batch_prompts*16, sampling_params)  # Generate for all nodes in a batch

        prm_inputs = []  # To collect all candidates for reward model evaluation
        mapping = []  # To map back reward model scores to corresponding nodes

        # Collect candidates for reward model evaluation
        for parent_node, parent_cum_logprob in current_level_nodes:
            for candidate in batch_responses:
                if parent_node + candidate.outputs[0].text + "\n" not in [prm_input[1] for prm_input in prm_inputs]:
                    new_node = parent_node + candidate.outputs[0].text + "\n"
                    cumulative_tokens = len(candidate.prompt_token_ids) + len(candidate.outputs[0].token_ids)
                    prm_inputs.append((new_node[:-1] + " " + step_tag, new_node, parent_cum_logprob, cumulative_tokens))
                    mapping.append(len(prm_inputs) - 1)  # Store the index for mapping back the score

        # Batch reward model evaluation
        prm_scores = eval_prm([prm_input for prm_input, _, _, _ in prm_inputs])
        next_level_nodes = []

        # Distribute candidates back to their parent nodes
        for idx, (_, node, parent_cum_logprob, cumulative_tokens) in enumerate(prm_inputs):
            score = prm_scores[idx]  # Get the corresponding score
            new_cum_logprob = parent_cum_logprob + score  # Update cumulative log probability

            # Check for completions and sufficient score
            if "```" in node or "\\boxed" in node.split("\n\n")[-1]:
                completed_paths.append((node, new_cum_logprob))
            elif score > math.log(0.8) and cumulative_tokens <= 2000:  # Threshold check
                next_level_nodes.append((node, new_cum_logprob))

        # Prune to keep only the top 'n' candidates based on cumulative log probability
        next_level_nodes.sort(key=lambda x: x[1], reverse=True)  # Sort nodes by their cumulative log probability
        current_level_nodes = next_level_nodes[:n]  # Keep only the top 'n' nodes
        current_level += 1

        # If we already have 'n' completed paths, no need to continue
        if len(completed_paths) >= n:
            break
        if not current_level_nodes or current_level > 20:
            if not completed_paths:
                messages = [
                    {
                        "role": "user",
                        "content": problem + tool_instruction
                    }
                ]

                base_prompt = tokenizer.apply_chat_template(
                    messages,
                    tokenize=False
                ) + "```python\n"

                raw_outputs = llm.generate([base_prompt]*8, tool_sampling_params)

                for response in raw_outputs:
                    completed_paths.append("```python\n" + response.outputs[0].text)
            break

    results = []
    answers = []

    for path in completed_paths:
        try:
            if "```python\n" in path:
                result_output, code_output = process_output(path)
                gc.collect()
                
                results.append(code_output)
                answers.append(code_output)
            else:
                raw_output = path[0].split("\n\n")[-1]
                result_output, code_output = process_output(raw_output)
                gc.collect()

                results.append(result_output)
                answers.append(result_output)

        except Exception as e:
            print(e)
            result_output, code_output = -1, -1
            logprob = -10000

            results.append(result_output)
            answers.append(result_output)

    total_results.append(results)
    total_answers.append(answers)

In [ ]:
import numpy as np
from collections import Counter

final_answers = []

for a, b in zip(total_answers, total_results):
    a = np.array(a)
    b = np.array(b)
    a[a < 0] = b[a < 0]

    pred = Counter(a.tolist()).most_common(2)

    try:
        ans = pred[0][0] if not pred[0][0] < 0 else pred[1][0]
    except:
        if len(a) == 1:
            ans = a[0]
        else:
            ans = 0

    final_answers.append(ans)
    print(ans)


In [ ]:
df['answer'] = final_answers

In [ ]:
df

In [ ]:
df[['id','answer']].to_csv("submission.csv", header=True, index=False)

In [ ]:
df[['id','answer']].head()

**Concluding Observations and Recommendations**

Throughout our exploration with the DeepSeekMath-7B RL-tuned model and the application of the Tree of Thought (ToT) approach, pivotal insights have emerged, shaping my understanding of the model's capabilities and its limitations:

1. **Performance Upper Bound**: Analysis of the model's performance on the MATH test dataset has revealed an apparent "upper bound" to its capabilities. This limitation suggests that even with optimal tuning and application of sophisticated reasoning frameworks like ToT, there may be a hard limit to the complexity of problems the model can handle effectively. Interestingly, this upper bound might also explain the high scores observed in other notebooks, which could be more a factor of fortunate seeds rather than superior model performance or methodology.

2. **Challenges with Prompt Engineering and Reliability**: The model has shown limited responsiveness to nuanced prompt engineering, restricting its utility in settings where fine-tuning through prompts is essential. Moreover, despite the theoretical promise of ToT in enhancing reliability through multiple reasoning pathways, it has not consistently elevated the model's performance to overcome complex challenges within the dataset.

3. **Recommendations for Further Development**:
   - **In-depth Analysis and Enhanced Training**: Deeper analysis into the model’s inherent limitations and targeted training may help in pushing its performance beyond the observed upper bound.
   - **Refinement of ToT**: Improving the ToT methodology by incorporating more robust evaluation metrics or additional supportive mechanisms could bolster its effectiveness.

**Future Directions**

Considering these insights, future work should aim at identifying and transcending the performance limitations of the current model. Efforts should focus on expanding the model's problem-solving capabilities through advanced training, refining reasoning methodologies, and potentially exploring new model architectures or hybrid approaches. The objective remains not only to surpass the current leaderboard scores but to expand the boundaries of AI capabilities in solving highly complex problems.